In [2]:
# -*- coding: utf-8 -*-
# pylint:disable=line-too-long
"""Featurization code. Copied from the colorml package to make the installation a bit less of a pain"""
from __future__ import absolute_import, print_function

import os
from pathlib import Path

import numpy as np
import openbabel as ob
import pandas as pd
import pybel
from mofid.run_mofid import cif2mofid
from pymatgen.io.cif import CifParser
from six.moves import zip

# This code relies on my fork of molsimplify which outputs the sum and the average RACs
from molSimplify.Informatics.MOF.MOF_descriptors import get_MOF_descriptors


import contextlib
import os
import shutil
import tempfile


@contextlib.contextmanager
def make_temp_directory():
    """Contextmanager that creates temp dir"""
    temp_dir = tempfile.mkdtemp()
    try:
        yield temp_dir
    finally:
        shutil.rmtree(temp_dir)


@contextlib.contextmanager
def temp():
    """Contextmanager that creates temp file"""
    tmp = tempfile.NamedTemporaryFile(delete=False)
    try:
        yield tmp
    finally:
        tmp.close()  # closes the file, so we can right remove it
        os.remove(tmp.name)


class FeaturizationException(Exception):
    pass


def get_primitive(datapath, writepath):
    """Generate primitive structure which is needed for molsimplify"""
    s = CifParser(datapath, occupancy_tolerance=1).get_structures()[0]  # pylint:disable=invalid-name
    sprim = s.get_primitive_structure()
    sprim.to('cif', writepath)


def openbabel_count_bond_order(mol, bo=2):  # pylint:disable=invalid-name
    """Count how often bond order bo occures in openbabel molecule mol"""
    count = 0
    mole = mol.OBMol
    for bond in ob.OBMolBondIter(mole):
        if bond.GetBO() == bo:
            count += 1
    return count


def openbabel_count_aromatic_rings(mol):
    """Calculate aromatic rings in an openbabel molecule"""
    count = 0
    mole = mol.OBMol
    for ring in mole.GetSSSR():
        # Note: the OB implementation is wrong. It assumes that if all
        # atoms in the ring are aromatic then the ring itself must be
        # aromatic. This is not necessarily true.
        if ring.IsAromatic():
            count += 1
    return count


def openbabel_count_aromatic(mol):
    """Calculate aromatic bonds in an openbabel molecule"""
    carboncounter = 1
    double_cc = 1
    mole = mol.OBMol
    for bond in ob.OBMolBondIter(mole):
        atom1 = bond.GetBeginAtom()
        atom2 = bond.GetEndAtom()
        symbol1 = atom1.GetAtomicNum()
        symbol2 = atom2.GetAtomicNum()

        if symbol1 == symbol2 == 6:
            carboncounter += 1
            if bond.IsDouble():
                double_cc += 1

    return double_cc / carboncounter - 1


def get_group_counts(mol):
    """Calculate the groups that are already implemented in openbabel based on an openbabel mol"""
    mole = mol.OBMol
    group_dict = {
        'primary_amide': 0,
        'secondary_amide': 0,
        'tertiary_amide': 0,
        'ester': 0,
        'carbonyl': 0,
    }
    for bond in ob.OBMolBondIter(mole):
        if bond.IsPrimaryAmide():
            group_dict['primary_amide'] += 1
        elif bond.IsSecondaryAmide():
            group_dict['primary_amide'] += 1
        elif bond.IsTertiaryAmide():
            group_dict['tertiary_amide'] += 1
        elif bond.IsEster():
            group_dict['ester'] += 1
        elif bond.IsCarbonyl():
            group_dict['carbonyl'] += 1

    return group_dict


def get_molecular_descriptors(smiles):
    """get heuristics from strings using openbabel"""
    mymol = pybel.readstring('smi', smiles)

    descriptordict = {}

    group_counts = get_group_counts(mymol)
    desc = mymol.calcdesc()
    db_ratio = openbabel_count_aromatic(mymol)
    aromatic_rings = openbabel_count_aromatic_rings(mymol)

    descriptordict.update(group_counts)
    descriptordict['logP'] = desc['logP']
    descriptordict['MR'] = desc['MR']
    descriptordict['dbratio'] = db_ratio
    descriptordict['aromatic_rings'] = aromatic_rings
    descriptordict['dbonds'] = desc['dbonds']
    descriptordict['abonds'] = desc['abonds']

    return descriptordict


def get_smiles_features(cif):
    """Use openbabel to calculate some features based on the smiles which we get from MOFid"""
    # make sure that the output is automatically deleted.
    with make_temp_directory() as temp_dir:
        mofid = cif2mofid(cif, temp_dir)
    name = mofid['cifname']

    linker_descriptors = []

    try:
        for linker in mofid['smiles_linkers']:
            linker_descriptors.append(list(get_molecular_descriptors(linker).values()))
            # super inefficient to do this all the time. But i do not know if i'll change the descriptorlist ...
            keys = list(get_molecular_descriptors(linker).keys())
            mean_keys = [s + '_mean' for s in keys]
            sum_keys = [s + '_sum' for s in keys]

        linker_descriptors = np.array(linker_descriptors)
        means = np.mean(linker_descriptors, axis=0)
        sums = np.mean(linker_descriptors, axis=0)

        mean_dict = dict(list(zip(mean_keys, means)))
        sum_dict = dict(list(zip(sum_keys, sums)))

        result_dict = {}
        result_dict['name'] = name

        result_dict.update(sum_dict)
        result_dict.update(mean_dict)

        return result_dict
    except Exception as e:  # pylint:disable=invalid-name, broad-except
        print(e)
        return None


def get_moldesc(cifs):
    """featurizers the linkers"""
    result_list = []
    for cif in cifs:
        result_list.append(get_smiles_features(cif))
    try:
        df = pd.DataFrame(result_list)  # pylint:disable=invalid-name
    except Exception as e:  # pylint:disable=broad-except
        print(e)
        df = result_list  # pylint:disable=invalid-name

    return df


def get_racs(cif):
    """Assumes that cif is primitive. As there is no way in molsimplify to stop writing stuff, we do it in some temporary directory"""
    featurization_list = []
    # need to catch and handle exceptions

    with make_temp_directory() as temp_dir:
        full_names, full_descriptors = get_MOF_descriptors(
            cif,  # inputstructure
            3,  # scope
            path=temp_dir,  # stuff will be dumped here
            xyzpath=os.path.join(temp_dir, 'file.xyz'),
        )
        full_names.append('filename')
        full_descriptors.append(Path(cif).stem)
        featurization = dict(zip(full_names, full_descriptors))
        featurization_list.append(featurization)

    df = pd.DataFrame(featurization_list)  # pylint:disable=invalid-name
    keep = [val for val in df.columns.values if ('mc' in val) or ('lc' in val) or ('f-lig' in val) or ('func' in val)]
    df = df[['filename'] + keep]  # pylint:disable=invalid-name

    # Now, we need to employ a workaround because the feature names changed mc_CR -> mc with open sourcing the MOF RACs code
    mc_features = [val for val in df.columns.values if 'mc' in val]
#     # if we use the old version, all should have CRY in name, in the new version no feature should have CRY in name
#     if not 'CRY' in mc_features[0]:
#         mc_features_new = [val.replace('mc', 'mc_CRY') for val in mc_features]
#         replacement_dict = dict(zip(mc_features, mc_features_new))
#         df.rename(columns=replacement_dict, inplace=True)
    # need to generate sums and differences
    return df


def merge_racs_moldesc(df_moldesc, df_racs):
    """Merge df assuming that the filename columns are filename and racs"""
    df_merged = pd.merge(df_racs, df_moldesc, left_on='filename', right_on='name')
    return df_merged


def get_color_descriptors(cif):
    """Orchestrate the featurization"""
    try:
        with temp() as tempfile:
            tempname = tempfile.name
            get_primitive(cif, tempname)
            moldesc = get_moldesc([tempname])
            racs = get_racs(tempname)
        df_features = merge_racs_moldesc(moldesc, racs)
        return df_features
    except Exception as e:  # pylint:disable=invalid-name
        raise FeaturizationException('Could not featurize the structure due to  {}'.format(e))


In [3]:
CHEMICAL_FEATURES = [
    'mc_CRY-chi-0-all',
    'mc_CRY-chi-1-all',
    'mc_CRY-chi-2-all',
    'mc_CRY-chi-3-all',
    'mc_CRY-Z-0-all',
    'mc_CRY-Z-1-all',
    'mc_CRY-Z-2-all',
    'mc_CRY-Z-3-all',
    'mc_CRY-I-1-all',
    'mc_CRY-I-2-all',
    'mc_CRY-I-3-all',
    'mc_CRY-T-0-all',
    'mc_CRY-T-1-all',
    'mc_CRY-T-2-all',
    'mc_CRY-T-3-all',
    'mc_CRY-S-0-all',
    'mc_CRY-S-1-all',
    'mc_CRY-S-2-all',
    'mc_CRY-S-3-all',
    'D_mc_CRY-chi-1-all',
    'D_mc_CRY-chi-2-all',
    'D_mc_CRY-chi-3-all',
    'D_mc_CRY-Z-1-all',
    'D_mc_CRY-Z-2-all',
    'D_mc_CRY-Z-3-all',
    'D_mc_CRY-T-1-all',
    'D_mc_CRY-T-2-all',
    'D_mc_CRY-T-3-all',
    'D_mc_CRY-S-1-all',
    'D_mc_CRY-S-2-all',
    'D_mc_CRY-S-3-all',
    'func-chi-0-all',
    'func-chi-1-all',
    'func-chi-2-all',
    'func-chi-3-all',
    'func-Z-0-all',
    'func-Z-1-all',
    'func-Z-2-all',
    'func-Z-3-all',
    'func-I-1-all',
    'func-I-2-all',
    'func-I-3-all',
    'func-T-0-all',
    'func-T-1-all',
    'func-T-2-all',
    'func-T-3-all',
    'func-S-0-all',
    'func-S-1-all',
    'func-S-2-all',
    'func-S-3-all',
    'func-alpha-0-all',
    'func-alpha-1-all',
    'func-alpha-2-all',
    'func-alpha-3-all',
    'D_func-chi-1-all',
    'D_func-chi-2-all',
    'D_func-chi-3-all',
    'D_func-Z-1-all',
    'D_func-Z-2-all',
    'D_func-Z-3-all',
    'D_func-T-1-all',
    'D_func-T-2-all',
    'D_func-T-3-all',
    'D_func-S-2-all',
    'D_func-S-3-all',
    'D_func-alpha-1-all',
    'D_func-alpha-2-all',
    'D_func-alpha-3-all',
    'f-lig-chi-0',
    'f-lig-chi-1',
    'f-lig-chi-2',
    'f-lig-chi-3',
    'f-lig-Z-0',
    'f-lig-Z-1',
    'f-lig-Z-2',
    'f-lig-Z-3',
    'f-lig-I-0',
    'f-lig-I-1',
    'f-lig-I-2',
    'f-lig-I-3',
    'f-lig-T-0',
    'f-lig-T-1',
    'f-lig-T-2',
    'f-lig-T-3',
    'f-lig-S-0',
    'f-lig-S-1',
    'f-lig-S-2',
    'f-lig-S-3',
    'lc-chi-0-all',
    'lc-chi-1-all',
    'lc-chi-2-all',
    'lc-chi-3-all',
    'lc-Z-0-all',
    'lc-Z-1-all',
    'lc-Z-2-all',
    'lc-Z-3-all',
    'lc-I-2-all',
    'lc-I-3-all',
    'lc-T-0-all',
    'lc-T-1-all',
    'lc-T-2-all',
    'lc-T-3-all',
    'lc-S-3-all',
    'lc-alpha-0-all',
    'lc-alpha-1-all',
    'lc-alpha-2-all',
    'lc-alpha-3-all',
    'D_lc-chi-2-all',
    'D_lc-chi-3-all',
    'D_lc-Z-1-all',
    'D_lc-Z-2-all',
    'D_lc-Z-3-all',
    'D_lc-T-1-all',
    'D_lc-T-2-all',
    'D_lc-T-3-all',
    'D_lc-alpha-1-all',
    'D_lc-alpha-2-all',
    'D_lc-alpha-3-all',
    'tertiary_amide_sum',
    'ester_sum',
    'carbonyl_sum',
    'logP_sum',
    'MR_sum',
    'aromatic_rings_sum',
    'dbonds_sum',
    'abonds_sum',
    'tertiary_amide_mean',
    'ester_mean',
    'carbonyl_mean',
    'logP_mean',
    'MR_mean',
    'aromatic_rings_mean',
    'dbonds_mean',
    'abonds_mean',
    'sum-func-chi-0-all',
    'sum-func-chi-1-all',
    'sum-func-chi-2-all',
    'sum-func-chi-3-all',
    'sum-func-Z-0-all',
    'sum-func-Z-1-all',
    'sum-func-Z-2-all',
    'sum-func-Z-3-all',
    'sum-func-I-0-all',
    'sum-func-I-1-all',
    'sum-func-I-2-all',
    'sum-func-I-3-all',
    'sum-func-T-0-all',
    'sum-func-T-1-all',
    'sum-func-T-2-all',
    'sum-func-T-3-all',
    'sum-func-S-0-all',
    'sum-func-S-1-all',
    'sum-func-S-2-all',
    'sum-func-S-3-all',
    'sum-func-alpha-0-all',
    'sum-func-alpha-1-all',
    'sum-func-alpha-2-all',
    'sum-func-alpha-3-all',
    'sum-D_func-chi-1-all',
    'sum-D_func-chi-2-all',
    'sum-D_func-chi-3-all',
    'sum-D_func-Z-1-all',
    'sum-D_func-Z-2-all',
    'sum-D_func-Z-3-all',
    'sum-D_func-T-1-all',
    'sum-D_func-T-2-all',
    'sum-D_func-T-3-all',
    'sum-D_func-S-1-all',
    'sum-D_func-S-2-all',
    'sum-D_func-S-3-all',
    'sum-D_func-alpha-1-all',
    'sum-D_func-alpha-2-all',
    'sum-D_func-alpha-3-all',
    'sum-f-lig-chi-0',
    'sum-f-lig-chi-1',
    'sum-f-lig-chi-2',
    'sum-f-lig-chi-3',
    'sum-f-lig-Z-0',
    'sum-f-lig-Z-1',
    'sum-f-lig-Z-2',
    'sum-f-lig-Z-3',
    'sum-f-lig-I-0',
    'sum-f-lig-I-1',
    'sum-f-lig-I-2',
    'sum-f-lig-I-3',
    'sum-f-lig-T-0',
    'sum-f-lig-T-1',
    'sum-f-lig-T-2',
    'sum-f-lig-T-3',
    'sum-f-lig-S-0',
    'sum-f-lig-S-1',
    'sum-f-lig-S-2',
    'sum-f-lig-S-3',
    'sum-lc-chi-0-all',
    'sum-lc-chi-1-all',
    'sum-lc-chi-2-all',
    'sum-lc-chi-3-all',
    'sum-lc-Z-0-all',
    'sum-lc-Z-1-all',
    'sum-lc-Z-2-all',
    'sum-lc-Z-3-all',
    'sum-lc-I-0-all',
    'sum-lc-I-1-all',
    'sum-lc-I-2-all',
    'sum-lc-I-3-all',
    'sum-lc-T-0-all',
    'sum-lc-T-1-all',
    'sum-lc-T-2-all',
    'sum-lc-T-3-all',
    'sum-lc-S-0-all',
    'sum-lc-S-1-all',
    'sum-lc-S-2-all',
    'sum-lc-S-3-all',
    'sum-lc-alpha-0-all',
    'sum-lc-alpha-1-all',
    'sum-lc-alpha-2-all',
    'sum-lc-alpha-3-all',
    'sum-D_lc-chi-1-all',
    'sum-D_lc-chi-2-all',
    'sum-D_lc-chi-3-all',
    'sum-D_lc-Z-1-all',
    'sum-D_lc-Z-2-all',
    'sum-D_lc-Z-3-all',
    'sum-D_lc-T-1-all',
    'sum-D_lc-T-2-all',
    'sum-D_lc-T-3-all',
    'sum-D_lc-S-1-all',
    'sum-D_lc-S-2-all',
    'sum-D_lc-S-3-all',
    'sum-D_lc-alpha-1-all',
    'sum-D_lc-alpha-2-all',
    'sum-D_lc-alpha-3-all',
    'sum-mc_CRY-chi-0-all',
    'sum-mc_CRY-chi-1-all',
    'sum-mc_CRY-chi-2-all',
    'sum-mc_CRY-chi-3-all',
    'sum-mc_CRY-Z-0-all',
    'sum-mc_CRY-Z-1-all',
    'sum-mc_CRY-Z-2-all',
    'sum-mc_CRY-Z-3-all',
    'sum-mc_CRY-I-0-all',
    'sum-mc_CRY-I-1-all',
    'sum-mc_CRY-I-2-all',
    'sum-mc_CRY-I-3-all',
    'sum-mc_CRY-T-0-all',
    'sum-mc_CRY-T-1-all',
    'sum-mc_CRY-T-2-all',
    'sum-mc_CRY-T-3-all',
    'sum-mc_CRY-S-0-all',
    'sum-mc_CRY-S-1-all',
    'sum-mc_CRY-S-2-all',
    'sum-mc_CRY-S-3-all',
    'sum-D_mc_CRY-chi-1-all',
    'sum-D_mc_CRY-chi-2-all',
    'sum-D_mc_CRY-chi-3-all',
    'sum-D_mc_CRY-Z-1-all',
    'sum-D_mc_CRY-Z-2-all',
    'sum-D_mc_CRY-Z-3-all',
    'sum-D_mc_CRY-T-1-all',
    'sum-D_mc_CRY-T-2-all',
    'sum-D_mc_CRY-T-3-all',
    'sum-D_mc_CRY-S-1-all',
    'sum-D_mc_CRY-S-2-all',
    'sum-D_mc_CRY-S-3-all',
]


def _featurize(cif):
    """Runs featurization and returns none in case of FeaturizationException"""
    try:
        descriptors = get_color_descriptors(cif)
        return descriptors[CHEMICAL_FEATURES]
    except FeaturizationException as e:
        print(e)
        return None

In [13]:
f = _featurize('/Users/kevinmaikjablonka/Dropbox (LSMO)/proj75_mofcolor/ml/data/case_studies/mof74/ORIVOC_clean.cif')

Created a new output directory: /var/folders/_0/7bjj2kkj1t7c22rrx8hwjpzm0000gn/T/tmpqrgkpp6t/MetalOxo
Created a new output directory: /var/folders/_0/7bjj2kkj1t7c22rrx8hwjpzm0000gn/T/tmpqrgkpp6t/SingleNode
Created a new output directory: /var/folders/_0/7bjj2kkj1t7c22rrx8hwjpzm0000gn/T/tmpqrgkpp6t/AllNode
Created a new output directory: /var/folders/_0/7bjj2kkj1t7c22rrx8hwjpzm0000gn/T/tmpqrgkpp6t/StandardIsolated
*** Open Babel Warning  in InChI code
  #0 :Accepted unusual valence(s): O(1);
*** Open Babel Warning  in InChI code
  #0 :Accepted unusual valence(s): O(1);
*** Open Babel Warning  in InChI code
  #0 :Accepted unusual valence(s): O(1);
*** Open Babel Warning  in InChI code
  #0 :Accepted unusual valence(s): O(1);
*** Open Babel Warning  in InChI code
  #0 :Accepted unusual valence(s): O(1);
*** Open Babel Warning  in InChI code
  #0 :Accepted unusual valence(s): O(1);
*** Open Babel Warning  in InChI code
  #0 :Accepted unusual valence(s): O(1);
*** Open Babel Warning  in InC

('cell vectors: ', 'alpha, beta, gamma = 117.93032814, 98.34592124 ,98.34592124')
[6.652, 0, 0]
[-2.2173333323064512, 15.114452692949614, 0]
[-2.2173333323064512, -7.55722634690183, 13.089499996145944]
Making the SBU RACs worked
looping over linkers
generating lc autocorrelations
looping over linkers
generating lc autocorrelations
looping over linkers
generating lc autocorrelations
312 312


In [5]:
f

,mc_CRY-chi-0-all,mc_CRY-chi-1-all,mc_CRY-chi-2-all,mc_CRY-chi-3-all,mc_CRY-Z-0-all,mc_CRY-Z-1-all,mc_CRY-Z-2-all,mc_CRY-Z-3-all,mc_CRY-I-1-all,mc_CRY-I-2-all,...,sum-D_mc_CRY-chi-3-all,sum-D_mc_CRY-Z-1-all,sum-D_mc_CRY-Z-2-all,sum-D_mc_CRY-Z-3-all,sum-D_mc_CRY-T-1-all,sum-D_mc_CRY-T-2-all,sum-D_mc_CRY-T-3-all,sum-D_mc_CRY-S-1-all,sum-D_mc_CRY-S-2-all,sum-D_mc_CRY-S-3-all
0,2.7225,22.704,22.275,45.21,900.0,960.0,2520.0,1920.0,4.0,6.0,...,-75.3,528.0,576.0,1236.0,36.0,24.0,78.0,13.92,12.96,30.36


In [3]:
from glob import glob
from pathlib import Path

In [4]:
all_structures = glob('/Users/kevinmaikjablonka/Dropbox (LSMO)/proj75_mofcolor/ml/data/case_studies/mof74/*.cif')

In [5]:
all_structures += glob('/Users/kevinmaikjablonka/Dropbox (LSMO)/proj75_mofcolor/ml/data/case_studies/chris/*.cif')

In [6]:
all_structures += glob('/Users/kevinmaikjablonka/Dropbox (LSMO)/proj75_mofcolor/ml/data/case_studies/olgas_structures/*.cif')

In [7]:
all_structures += glob('/Users/kevinmaikjablonka/Dropbox (LSMO)/proj75_mofcolor/ml/data/case_studies/olgas_structures/*.cif')

In [8]:
all_structures += glob('/Users/kevinmaikjablonka/Dropbox (LSMO)/proj75_mofcolor/ml/data/case_studies/mofs/*.cif')

In [9]:
all_structures

['/Users/kevinmaikjablonka/Dropbox (LSMO)/proj75_mofcolor/ml/data/case_studies/mof74/NAVJAW_clean.cif',
 '/Users/kevinmaikjablonka/Dropbox (LSMO)/proj75_mofcolor/ml/data/case_studies/mof74/ORIVOC_clean.cif',
 '/Users/kevinmaikjablonka/Dropbox (LSMO)/proj75_mofcolor/ml/data/case_studies/mof74/VOGTIV.cif',
 '/Users/kevinmaikjablonka/Dropbox (LSMO)/proj75_mofcolor/ml/data/case_studies/mof74/ORIVUI_clean.cif',
 '/Users/kevinmaikjablonka/Dropbox (LSMO)/proj75_mofcolor/ml/data/case_studies/chris/sion13.cif',
 '/Users/kevinmaikjablonka/Dropbox (LSMO)/proj75_mofcolor/ml/data/case_studies/chris/UiO66-NH2.cif',
 '/Users/kevinmaikjablonka/Dropbox (LSMO)/proj75_mofcolor/ml/data/case_studies/chris/sion24.cif',
 '/Users/kevinmaikjablonka/Dropbox (LSMO)/proj75_mofcolor/ml/data/case_studies/olgas_structures/UiO-66-NH2.cif',
 '/Users/kevinmaikjablonka/Dropbox (LSMO)/proj75_mofcolor/ml/data/case_studies/olgas_structures/UiO-66-NH2-W.cif',
 '/Users/kevinmaikjablonka/Dropbox (LSMO)/proj75_mofcolor/ml/data

In [10]:
test_structures = {}


for s in all_structures: 
    test_structures[Path(s).stem] = _featurize(s)

Created a new output directory: /var/folders/_0/7bjj2kkj1t7c22rrx8hwjpzm0000gn/T/tmpt6phu2wc/MetalOxo
Created a new output directory: /var/folders/_0/7bjj2kkj1t7c22rrx8hwjpzm0000gn/T/tmpt6phu2wc/SingleNode
Created a new output directory: /var/folders/_0/7bjj2kkj1t7c22rrx8hwjpzm0000gn/T/tmpt6phu2wc/AllNode
Created a new output directory: /var/folders/_0/7bjj2kkj1t7c22rrx8hwjpzm0000gn/T/tmpt6phu2wc/StandardIsolated
*** Open Babel Warning  in InChI code
  #0 :Accepted unusual valence(s): O(1);
*** Open Babel Warning  in InChI code
  #0 :Accepted unusual valence(s): O(1);
*** Open Babel Warning  in InChI code
  #0 :Accepted unusual valence(s): O(1);
*** Open Babel Warning  in InChI code
  #0 :Accepted unusual valence(s): O(1);
*** Open Babel Warning  in InChI code
  #0 :Accepted unusual valence(s): O(1);
*** Open Babel Warning  in InChI code
  #0 :Accepted unusual valence(s): O(1);
*** Open Babel Warning  in InChI code
  #0 :Accepted unusual valence(s): O(1);
*** Open Babel Warning  in InC

('cell vectors: ', 'alpha, beta, gamma = 117.87883596, 98.45077411 ,98.451')
[6.7192, 0, 0]
[-2.2397537362802376, 15.074720536076903, 0]
[-2.23969252706955, -7.537357137712095, 13.055089133930855]
Making the SBU RACs worked
looping over linkers
generating lc autocorrelations
looping over linkers
generating lc autocorrelations
looping over linkers
generating lc autocorrelations
312 312


Created a new output directory: /var/folders/_0/7bjj2kkj1t7c22rrx8hwjpzm0000gn/T/tmpacr7drnp/MetalOxo
Created a new output directory: /var/folders/_0/7bjj2kkj1t7c22rrx8hwjpzm0000gn/T/tmpacr7drnp/SingleNode
Created a new output directory: /var/folders/_0/7bjj2kkj1t7c22rrx8hwjpzm0000gn/T/tmpacr7drnp/AllNode
Created a new output directory: /var/folders/_0/7bjj2kkj1t7c22rrx8hwjpzm0000gn/T/tmpacr7drnp/StandardIsolated
*** Open Babel Warning  in InChI code
  #0 :Accepted unusual valence(s): O(1);
*** Open Babel Warning  in InChI code
  #0 :Accepted unusual valence(s): O(1);
*** Open Babel Warning  in InChI code
  #0 :Accepted unusual valence(s): O(1);
*** Open Babel Warning  in InChI code
  #0 :Accepted unusual valence(s): O(1);
*** Open Babel Warning  in InChI code
  #0 :Accepted unusual valence(s): O(1);
*** Open Babel Warning  in InChI code
  #0 :Accepted unusual valence(s): O(1);
*** Open Babel Warning  in InChI code
  #0 :Accepted unusual valence(s): O(1);
*** Open Babel Warning  in InC

('cell vectors: ', 'alpha, beta, gamma = 117.93032814, 98.34592124 ,98.34592124')
[6.652, 0, 0]
[-2.2173333323064512, 15.114452692949614, 0]
[-2.2173333323064512, -7.55722634690183, 13.089499996145944]
Making the SBU RACs worked
looping over linkers
generating lc autocorrelations
looping over linkers
generating lc autocorrelations
looping over linkers
generating lc autocorrelations
312 312


Created a new output directory: /var/folders/_0/7bjj2kkj1t7c22rrx8hwjpzm0000gn/T/tmpx7hbozhq/MetalOxo
Created a new output directory: /var/folders/_0/7bjj2kkj1t7c22rrx8hwjpzm0000gn/T/tmpx7hbozhq/SingleNode
Created a new output directory: /var/folders/_0/7bjj2kkj1t7c22rrx8hwjpzm0000gn/T/tmpx7hbozhq/AllNode
Created a new output directory: /var/folders/_0/7bjj2kkj1t7c22rrx8hwjpzm0000gn/T/tmpx7hbozhq/StandardIsolated
*** Open Babel Warning  in InChI code
  #0 :Accepted unusual valence(s): O(1);
*** Open Babel Warning  in InChI code
  #0 :Accepted unusual valence(s): O(1);
*** Open Babel Warning  in InChI code
  #0 :Accepted unusual valence(s): O(1);
*** Open Babel Warning  in InChI code
  #0 :Accepted unusual valence(s): O(1);
*** Open Babel Warning  in InChI code
  #0 :Accepted unusual valence(s): O(1);
*** Open Babel Warning  in InChI code
  #0 :Accepted unusual valence(s): O(1);
*** Open Babel Warning  in InChI code
  #0 :Accepted unusual valence(s): O(1);
*** Open Babel Warning  in InC

...

('cell vectors: ', 'alpha, beta, gamma = 89.99975933, 120.0001 ,120.00041684')
[18.5949, 0, 0]
[-9.297584317777133, 16.103617861180716, 0]
[-9.297528106318486, -5.367930352512864, 15.18273097445874]
Making the SBU RACs worked
looping over linkers
generating lc autocorrelations
looping over linkers
generating lc autocorrelations
looping over linkers
generating lc autocorrelations
looping over linkers
generating lc autocorrelations
looping over linkers
generating lc autocorrelations
looping over linkers
generating lc autocorrelations
looping over linkers
generating lc autocorrelations
looping over linkers
generating lc autocorrelations
312 312


Created a new output directory: /var/folders/_0/7bjj2kkj1t7c22rrx8hwjpzm0000gn/T/tmp0n5gp0_y/MetalOxo
Created a new output directory: /var/folders/_0/7bjj2kkj1t7c22rrx8hwjpzm0000gn/T/tmp0n5gp0_y/SingleNode
Created a new output directory: /var/folders/_0/7bjj2kkj1t7c22rrx8hwjpzm0000gn/T/tmp0n5gp0_y/AllNode
Created a new output directory: /var/folders/_0/7bjj2kkj1t7c22rrx8hwjpzm0000gn/T/tmp0n5gp0_y/StandardIsolated
*** Open Babel Warning  in InChI code
  #0 :Accepted unusual valence(s): N(4);
*** Open Babel Warning  in InChI code
  #0 :Accepted unusual valence(s): N(4);
*** Open Babel Warning  in InChI code
  #0 :Accepted unusual valence(s): N(4);
*** Open Babel Warning  in InChI code
  #0 :Accepted unusual valence(s): N(4);
*** Open Babel Warning  in InChI code
  #0 :Accepted unusual valence(s): N(4);
*** Open Babel Warning  in InChI code
  #0 :Accepted unusual valence(s): N(4);
*** Open Babel Warning  in InChI code
  #0 :Accepted unusual valence(s): N(4);
*** Open Babel Warning  in InC

('cell vectors: ', 'alpha, beta, gamma = 90.0, 93.7494 ,90.0')
[9.9494, 0, 0]
[1.984007416660657e-15, 32.4013, 0]
[-2.545096171383732, 2.5390171115991048e-15, 38.83689551854535]
Making the SBU RACs worked
looping over linkers
generating lc autocorrelations
looping over linkers
generating lc autocorrelations
looping over linkers
generating lc autocorrelations
looping over linkers
generating lc autocorrelations
looping over linkers
generating lc autocorrelations
looping over linkers
generating lc autocorrelations
looping over linkers
generating lc autocorrelations
looping over linkers
generating lc autocorrelations
looping over linkers
generating lc autocorrelations
looping over linkers
generating lc autocorrelations
looping over linkers
generating lc autocorrelations
looping over linkers
generating lc autocorrelations
looping over linkers
generating lc autocorrelations
looping over linkers
generating lc autocorrelations
looping over linkers
generating lc autocorrelations
looping over li

Created a new output directory: /var/folders/_0/7bjj2kkj1t7c22rrx8hwjpzm0000gn/T/tmpctw960d7/MetalOxo
Created a new output directory: /var/folders/_0/7bjj2kkj1t7c22rrx8hwjpzm0000gn/T/tmpctw960d7/SingleNode
Created a new output directory: /var/folders/_0/7bjj2kkj1t7c22rrx8hwjpzm0000gn/T/tmpctw960d7/AllNode
Created a new output directory: /var/folders/_0/7bjj2kkj1t7c22rrx8hwjpzm0000gn/T/tmpctw960d7/StandardIsolated


('cell vectors: ', 'alpha, beta, gamma = 60.24752823, 60.19712935 ,60.09991696')
[16.19242963, 0, 0]
[8.074224300628403, 14.04145932378103, 0]
[8.064788836414834, 4.651298100608696, 13.28983409904776]
Making the SBU RACs worked
looping over linkers
generating lc autocorrelations
looping over linkers
generating lc autocorrelations
looping over linkers
generating lc autocorrelations
looping over linkers
generating lc autocorrelations
looping over linkers
generating lc autocorrelations
looping over linkers
generating lc autocorrelations
312 312


In [11]:
merged_data = pd.concat([v for _, v in test_structures.items()])

In [12]:
merged_data.to_csv('test_set.csv', index=False)

In [7]:
df = pd.read_csv('ml/data/color_feat_merged.csv')

In [22]:
np.where(np.abs(df[CHEMICAL_FEATURES].values - mof74s['ORIVOC_clean'][CHEMICAL_FEATURES].values).sum(axis=1) < 0.01)

(array([4444]),)

In [23]:
df.iloc[4444]['refcode']

'WOBHEB'

In [25]:
df[CHEMICAL_FEATURES].drop_duplicates()

,mc_CRY-chi-0-all,mc_CRY-chi-1-all,mc_CRY-chi-2-all,mc_CRY-chi-3-all,mc_CRY-Z-0-all,mc_CRY-Z-1-all,mc_CRY-Z-2-all,mc_CRY-Z-3-all,mc_CRY-I-1-all,mc_CRY-I-2-all,...,sum-D_mc_CRY-chi-3-all,sum-D_mc_CRY-Z-1-all,sum-D_mc_CRY-Z-2-all,sum-D_mc_CRY-Z-3-all,sum-D_mc_CRY-T-1-all,sum-D_mc_CRY-T-2-all,sum-D_mc_CRY-T-3-all,sum-D_mc_CRY-S-1-all,sum-D_mc_CRY-S-2-all,sum-D_mc_CRY-S-3-all
0,1.904400,28.251360,41.576640,53.599200,8464.00,18179.200000,15897.600000,30102.40,7.200000,10.400000,...,-200.72,4648.0,7840.0,9240.0,248.0,416.0,472.0,51.68,82.56,100.72
1,1.210000,18.920000,11.220000,22.572000,3249.00,2280.000000,1368.000000,2736.00,5.000000,4.000000,...,-51.28,980.0,816.0,1404.0,60.0,32.0,72.0,19.20,14.72,26.24
2,0.672400,13.163733,15.025133,46.018400,361.00,709.333333,2115.333333,2394.00,4.666667,9.666667,...,-473.36,616.0,868.0,3124.0,60.0,92.0,424.0,68.88,81.24,303.72
3,1.037450,21.094400,20.642338,52.358200,2173.00,2191.000000,4921.375000,4343.50,6.000000,8.375000,...,-494.38,3760.0,5744.0,13404.0,392.0,450.0,1434.0,86.72,99.84,290.10
4,3.153675,17.078300,21.177937,37.109187,2680.75,1240.500000,1374.250000,2252.75,3.125000,4.750000,...,-112.76,2246.0,3442.0,5796.0,40.0,24.0,100.0,33.58,47.74,87.32
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6395,5.108425,34.778400,20.158675,86.624362,4659.25,2482.000000,17679.375000,6179.75,4.375000,3.750000,...,-776.80,17616.0,984.0,44136.0,728.0,-24.0,1848.0,198.64,12.24,497.12
6396,3.145975,29.317000,32.018275,79.484200,864.75,1114.750000,1947.500000,2588.00,5.000000,8.000000,...,-231.75,1272.0,1920.0,3981.0,156.0,234.0,618.0,30.78,46.02,107.55
6397,3.610000,21.660000,28.462000,50.730000,841.00,3480.000000,2436.000000,4524.00,4.000000,6.000000,...,-61.60,-32.0,720.0,1072.0,48.0,32.0,80.0,-0.48,19.84,28.80
6398,2.656900,33.643200,19.282900,39.054800,529.00,1104.000000,1081.000000,1288.00,6.000000,5.000000,...,-43.68,360.0,272.0,512.0,96.0,48.0,112.0,11.76,7.20,15.04


In [ ]:
np.where(np.abs(df[CHEMICAL_FEATURES].values - mof74s[''][CHEMICAL_FEATURES].values).sum(1) < .1)

In [ ]:
mof74s['ORIVOC_clean'][CHEMICAL_FEATURES]

Test that we can find HKUST1.

In [4]:
hkust = _featurize('./ml/data/case_studies/famous_mofs/AlPMOF.cif')

Created a new output directory: /var/folders/_0/7bjj2kkj1t7c22rrx8hwjpzm0000gn/T/tmpgzf0dttt/MetalOxo
Created a new output directory: /var/folders/_0/7bjj2kkj1t7c22rrx8hwjpzm0000gn/T/tmpgzf0dttt/SingleNode
Created a new output directory: /var/folders/_0/7bjj2kkj1t7c22rrx8hwjpzm0000gn/T/tmpgzf0dttt/AllNode
Created a new output directory: /var/folders/_0/7bjj2kkj1t7c22rrx8hwjpzm0000gn/T/tmpgzf0dttt/StandardIsolated
*** Open Babel Warning  in InChI code
  #0 :Accepted unusual valence(s): N(4);
*** Open Babel Warning  in InChI code
  #0 :Accepted unusual valence(s): N(4);
*** Open Babel Warning  in InChI code
  #0 :Accepted unusual valence(s): N(4);
*** Open Babel Warning  in InChI code
  #0 :Accepted unusual valence(s): N(4);
*** Open Babel Warning  in InChI code
  #0 :Accepted unusual valence(s): N(4);


('cell vectors: ', 'alpha, beta, gamma = 90.0, 90.0 ,101.62932666')
[6.5812, 0, 0]
[-3.290600001002629, 15.988999998244052, 0]
[1.0324997163611333e-15, 1.266631429644987e-15, 16.862]
Failed to featurize this MOF.
Could not featurize the structure due to  argument of type 'int' is not iterable


In [7]:
df[CHEMICAL_FEATURES].values  == hkust[CHEMICAL_FEATURES].values

array([[False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       ...,
       [False, False, False, ..., False, False, False],
       [ True,  True, False, ...,  True, False, False],
       [False, False, False, ..., False, False, False]])

In [8]:
np.where(np.abs(df[CHEMICAL_FEATURES].values - hkust.values).sum(1) < .1)

(array([6398]),)

In [9]:
hkust['sum-lc-chi-0-all']

0    47.3344
Name: sum-lc-chi-0-all, dtype: float64

In [10]:
df[df['refcode']=='ASOHUL']['sum-lc-chi-0-all']

6398    47.3344
Name: sum-lc-chi-0-all, dtype: float64

In [38]:
df_matcloud = pd.read_csv('/Users/kevinmaikjablonka/Downloads/data_paper/features_labels/CoRE2019_alldata.csv')

In [42]:
df_matcloud[df_matcloud['MOFname']=='ASOHUL_clean']['sum-lc-S-3-all']

9481    4.4968
Name: sum-lc-S-3-all, dtype: float64